In [ ]:
#A Study on Efficiency, Accuracy and Document Structure for Answer Sentence Selection
import numpy as np
import pandas as pd
QUESTION_LEN = 23
CANDIDATE_LEN = 64

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive


## Number Batch

In [ ]:
"""
This Python module provides just the code from the 'conceptnet5' module that
you need to represent terms, possibly with multiple words, as ConceptNet URIs.

It depends on 'wordfreq', a Python 3 library, so it can tokenize multilingual
text consistently: https://pypi.org/project/wordfreq/

Example:

>>> standardized_uri('es', 'ayudar')
'/c/es/ayudar'
>>> standardized_uri('en', 'a test phrase')
'/c/en/test_phrase'
>>> standardized_uri('en', '24 hours')
'/c/en/##_hours'
"""
!pip install wordfreq
import wordfreq
import re


# English-specific stopword handling
STOPWORDS = ['the', 'a', 'an']
DROP_FIRST = ['to']
DOUBLE_DIGIT_RE = re.compile(r'[0-9][0-9]')
DIGIT_RE = re.compile(r'[0-9]')


def standardized_uri(language, term):
    """
    Get a URI that is suitable to label a row of a vector space, by making sure
    that both ConceptNet's and word2vec's normalizations are applied to it.

    'language' should be a BCP 47 language code, such as 'en' for English.

    If the term already looks like a ConceptNet URI, it will only have its
    sequences of digits replaced by #. Otherwise, it will be turned into a
    ConceptNet URI in the given language, and then have its sequences of digits
    replaced.
    """
    if not (term.startswith('/') and term.count('/') >= 2):
        term = _standardized_concept_uri(language, term)
    return replace_numbers(term)


def english_filter(tokens):
    """
    Given a list of tokens, remove a small list of English stopwords. This
    helps to work with previous versions of ConceptNet, which often provided
    phrases such as 'an apple' and assumed they would be standardized to
	'apple'.
    """
    non_stopwords = [token for token in tokens if token not in STOPWORDS]
    while non_stopwords and non_stopwords[0] in DROP_FIRST:
        non_stopwords = non_stopwords[1:]
    if non_stopwords:
        return non_stopwords
    else:
        return tokens


def replace_numbers(s):
    """
    Replace digits with # in any term where a sequence of two digits appears.

    This operation is applied to text that passes through word2vec, so we
    should match it.
    """
    if DOUBLE_DIGIT_RE.search(s):
        return DIGIT_RE.sub('#', s)
    else:
        return s


def _standardized_concept_uri(language, term):
    if language == 'en':
        token_filter = english_filter
    else:
        token_filter = None
    language = language.lower()
    norm_text = _standardized_text(term, token_filter)
    return '/c/{}/{}'.format(language, norm_text)
    # return ''.format(language, norm_text)


def _standardized_text(text, token_filter):
    tokens = simple_tokenize(text.replace('_', ' '))
    if token_filter is not None:
        tokens = token_filter(tokens)
    return '_'.join(tokens)


def simple_tokenize(text):
    """
    Tokenize text using the default wordfreq rules.
    """
    return wordfreq.tokenize(text, 'xx')



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
embedding_dict = {}
word_dict={}
limit = 0
with open("numberbatch-en.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embedding_dict[word] = vector

In [ ]:
#given a word, standardise it and return the embedding
def get_embedding(word):
    word = standardized_uri('en', word)
    #truncate first 6 characters
    word = word[6:]
    if word in embedding_dict:
        return embedding_dict[word]
    else:
        return np.ones(300, dtype=np.float32)

## DATA

In [ ]:
df_train = pd.read_csv('WikiQACorpus/WikiQA-train.tsv', sep='\t')
df_train.head(30)

df_test = pd.read_csv('WikiQACorpus/WikiQA-test.tsv', sep='\t')
df_test.head(30)

df_dev = pd.read_csv('WikiQACorpus/WikiQA-dev.tsv', sep='\t')
df_dev.head(30)

print("train size : ",df_train.shape)
print("test size : ",df_test.shape)
print("dev size : ",df_dev.shape)


train size :  (20347, 7)
test size :  (6116, 7)
dev size :  (2733, 7)


In [ ]:
# for each Question, sentence, remove special characters

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

df_train['Question'] = df_train['Question'].apply(lambda x: clean_text(x))
df_train['Sentence'] = df_train['Sentence'].apply(lambda x: clean_text(x))

df_dev['Question'] = df_dev['Question'].apply(lambda x: clean_text(x))
df_dev['Sentence'] = df_dev['Sentence'].apply(lambda x: clean_text(x))

df_test['Question'] = df_test['Question'].apply(lambda x: clean_text(x))
df_test['Sentence'] = df_test['Sentence'].apply(lambda x: clean_text(x))

In [ ]:
#for each question, find the number of the words in the question
def get_question_length(row):
    return len(row['Question'].split())

def get_sentence_length(row):
    return len(row['Sentence'].split())

df_train['Question_length'] = df_train.apply(get_question_length, axis=1)
df_train['Candidate_length'] = df_train.apply(get_sentence_length, axis=1)
df_train = df_train.drop(['DocumentID', 'DocumentTitle', 'SentenceID'], axis=1)

df_test['Question_length'] = df_test.apply(get_question_length, axis=1)
df_test['Candidate_length'] = df_test.apply(get_sentence_length, axis=1)
df_test = df_test.drop(['DocumentID', 'DocumentTitle', 'SentenceID'], axis=1)

df_dev['Question_length'] = df_dev.apply(get_question_length, axis=1)
df_dev['Candidate_length'] = df_dev.apply(get_sentence_length, axis=1)
df_dev = df_dev.drop(['DocumentID', 'DocumentTitle', 'SentenceID'], axis=1)


In [ ]:
# find the maximum length of the question and the candidate sentence
max_question_length = df_train['Question_length'].max()
max_candidate_length = df_train['Candidate_length'].max()
#find the number of unique questions 
unique_questions = df_train['QuestionID'].unique()
print("Max question length in train data set: ", max_question_length)
print("Max candidate length in train data set: ", max_candidate_length)
print("Number of unique questions in train data set: ", len(unique_questions))

total_candidates = len(df_train)
print("Total number of candidates in train data set: ", total_candidates)

Max question length in train data set:  23
Max candidate length in train data set:  305
Number of unique questions in train data set:  2117
Total number of candidates in train data set:  20347


In [ ]:
# delete the row if sentence length is greater than 50 and label is 0

df_train = df_train.drop(df_train[(df_train['Label'] == 0) & (df_train['Candidate_length'] > CANDIDATE_LEN)].index)
df_test = df_test.drop(df_test[(df_test['Label'] == 0) & (df_test['Candidate_length'] > CANDIDATE_LEN)].index)
df_dev = df_dev.drop(df_dev[(df_dev['Label'] == 0) & (df_dev['Candidate_length'] > CANDIDATE_LEN)].index)



print("train size : ",df_train.shape)
print("test size : ",df_test.shape)
print("dev size : ",df_dev.shape)


train size :  (20251, 6)
test size :  (6092, 6)
dev size :  (2722, 6)


In [ ]:
QuestionID_greater_than_CANDIDATE_LEN = []

# for candidate if length is greater than CANDIDATE_LEN, add the question id to the list
for index, row in df_train.iterrows():
    if row['Candidate_length'] > CANDIDATE_LEN:
        QuestionID_greater_than_CANDIDATE_LEN.append(row['QuestionID'])

# delete the rows where the question id is in the list
df_train = df_train[~df_train['QuestionID'].isin(QuestionID_greater_than_CANDIDATE_LEN)]

#print the nubmer of unique questions
unique_questions = df_train['QuestionID'].unique()
print("Number of unique questions in train data set: ", len(unique_questions))


QuestionID_greater_than_CANDIDATE_LEN = []

# for candidate if length is greater than CANDIDATE_LEN, add the question id to the list
for index, row in df_test.iterrows():
    if row['Candidate_length'] > CANDIDATE_LEN:
        QuestionID_greater_than_CANDIDATE_LEN.append(row['QuestionID'])

# delete the rows where the question id is in the list
df_test = df_test[~df_test['QuestionID'].isin(QuestionID_greater_than_CANDIDATE_LEN)]

#print the nubmer of unique questions
unique_questions = df_test['QuestionID'].unique()
print("Number of unique questions in test data set: ", len(unique_questions))

QuestionID_greater_than_CANDIDATE_LEN = []

# for candidate if length is greater than CANDIDATE_LEN, add the question id to the list
for index, row in df_dev.iterrows():
    if row['Candidate_length'] > CANDIDATE_LEN:
        QuestionID_greater_than_CANDIDATE_LEN.append(row['QuestionID'])

# delete the rows where the question id is in the list
df_dev = df_dev[~df_dev['QuestionID'].isin(QuestionID_greater_than_CANDIDATE_LEN)]

#print the nubmer of unique questions
unique_questions = df_dev['QuestionID'].unique()
print("Number of unique questions in dev data set: ", len(unique_questions))




Number of unique questions in train data set:  2112
Number of unique questions in test data set:  628
Number of unique questions in dev data set:  295


In [ ]:
# dind the maximum length of the question and the candidate sentence
max_question_length = df_train['Question_length'].max()
max_candidate_length = df_train['Candidate_length'].max()

print("Max question length in train dataset: ", max_question_length)
print("Max candidate length in train dataset: ", max_candidate_length)

# dind the maximum length of the question and the candidate sentence
max_question_length = df_test['Question_length'].max()
max_candidate_length = df_test['Candidate_length'].max()

print("Max question length in test dataset: ", max_question_length)
print("Max candidate length in test dataset: ", max_candidate_length)

# dind the maximum length of the question and the candidate sentence
max_question_length = df_dev['Question_length'].max()
max_candidate_length = df_dev['Candidate_length'].max()

print("Max question length in dev dataset: ", max_question_length)
print("Max candidate length in dev dataset: ", max_candidate_length)

Max question length in train dataset:  23
Max candidate length in train dataset:  64
Max question length in test dataset:  19
Max candidate length in test dataset:  64
Max question length in dev dataset:  21
Max candidate length in dev dataset:  63


In [ ]:
# reformat the data where each question has a list of candidate sentences and a list of labels
# 1 if the sentence is the answer, 0 otherwise
def reformat_data(df):
    questions = []
    candidates = []
    labels = []
    for index, row in df.iterrows():
        if row['Question'] not in questions:
            questions.append(row['Question'])
            candidates.append([row['Sentence']])
            labels.append([row['Label']])
        else:
            candidates[questions.index(row['Question'])].append(row['Sentence'])
            labels[questions.index(row['Question'])].append(row['Label'])
    return questions, candidates, labels
    


In [ ]:
text_questions_train, text_candidates_train, labels_train = reformat_data(df_train)
text_questions_test, text_candidates_test, labels_test = reformat_data(df_test)
text_questions_dev, text_candidates_dev, labels_dev = reformat_data(df_dev)

max_question_length = QUESTION_LEN
max_candidate_length = CANDIDATE_LEN
print(max_question_length)
print(max_candidate_length)

23
64


In [ ]:
emb_questions_train = []
emb_candidates_train = []

# get the embedding for each word in the question and candidate sentence using the get_embedding function
for question in text_questions_train:
    emb_questions_train.append([get_embedding(word) for word in question.split()])
for candidate in text_candidates_train:
    emb_candidates_train.append([[get_embedding(word) for word in sentence.split()] for sentence in candidate])

emb_questions_test = []
emb_candidates_test = []

# get the embedding for each word in the question and candidate sentence using the get_embedding function
for question in text_questions_test:
    emb_questions_test.append([get_embedding(word) for word in question.split()])
for candidate in text_candidates_test:
    emb_candidates_test.append([[get_embedding(word) for word in sentence.split()] for sentence in candidate])

emb_questions_dev = []
emb_candidates_dev = []

# get the embedding for each word in the question and candidate sentence using the get_embedding function
for question in text_questions_dev:
    emb_questions_dev.append([get_embedding(word) for word in question.split()])
for candidate in text_candidates_dev:
    emb_candidates_dev.append([[get_embedding(word) for word in sentence.split()] for sentence in candidate])


In [ ]:
# print the total number of candidates for all questions
total_candidates_train = 0
for candidate in emb_candidates_train:
    total_candidates_train += len(candidate)
print("Total number of candidates train dataset: ", total_candidates_train)

print("Total number of questions train dataset: ", len(emb_questions_train))

Total number of candidates train dataset:  20220
Total number of questions train dataset:  2112


In [ ]:
# if all the labels are zero, remove the question and candidate sentence
def remove_all_zeros(labels,emb_questions,emb_candidates):
  for i in range(len(labels)):
      if sum(labels[i]) == 0:
          emb_questions[i] = []
          emb_candidates[i] = []
          labels[i] = []

  # remove the empty lists
  return [x for x in emb_questions if x != []] ,[x for x in emb_candidates if x != []] ,[x for x in labels if x != []]

emb_questions_train,emb_candidates_train,labels_train = remove_all_zeros(labels_train,emb_questions_train,emb_candidates_train)
emb_questions_test,emb_candidates_test,labels_test = remove_all_zeros(labels_test,emb_questions_test,emb_candidates_test)
emb_questions_dev,emb_candidates_dev,labels_dev = remove_all_zeros(labels_dev,emb_questions_dev,emb_candidates_dev)

# print the total number of candidates for all questions
total_candidates = 0
for candidate in emb_candidates_train:
    total_candidates += len(candidate)
print("Total number of candidates in train dataset: ", total_candidates)

print("Total number of questions in train dataset: ", len(emb_questions_train))

Total number of candidates in train dataset:  8592
Total number of questions in train dataset:  867


In [ ]:
# for each question, candiate sentence pair, perform a cosinet between each embedding of question to each embedding of sentence to identify the most similar word

def get_cosine_similarity(emb_question, emb_candidate):
    question_cosines = []
    candidate_cosines = []
    # for each word in question, find the word in the candidate sentence that is most similar and calculate the maximum cosine similarity
    for word in emb_question:
        question_cosines.append(max([np.dot(word, candidate_word)/(np.linalg.norm(word)*np.linalg.norm(candidate_word)) for candidate_word in emb_candidate]))
        

    for word in emb_candidate:
        candidate_cosines.append(max([np.dot(word, question_word)/(np.linalg.norm(word)*np.linalg.norm(question_word)) for question_word in emb_question]))
        

    return question_cosines, candidate_cosines

In [ ]:
import copy

def embeddings_q_c_l(emb_questions,emb_candidates,labels):
  r_emb_question_candidate_pairs = []


  # for each question, candidate sentence pair, get the cosine similarity between each word in the question and each word in the candidate sentence
  # append the cosine similarity values to each question and candidate sentence embedding
  for i in range(len(emb_questions)):
      temp = []
      for j in range(len(emb_candidates[i])):
          question_cosines, candidate_cosines = get_cosine_similarity(emb_questions[i], emb_candidates[i][j])

          # take a deep copy of emb_question[i]
          temp_emb_question = copy.deepcopy(emb_questions[i])
          temp_emb_candidate = copy.deepcopy(emb_candidates[i][j])
          for k in range(len(temp_emb_question)):
              temp_emb_question[k] = np.append(temp_emb_question[k], question_cosines[k])
          for k in range(len(temp_emb_candidate)):
              temp_emb_candidate[k] = np.append(temp_emb_candidate[k], candidate_cosines[k])
          temp.append([temp_emb_question, temp_emb_candidate, labels[i][j]])
      r_emb_question_candidate_pairs.append(temp)
  return r_emb_question_candidate_pairs

r_emb_question_candidate_pairs_train = embeddings_q_c_l(emb_questions_train,emb_candidates_train,labels_train)
r_emb_question_candidate_pairs_test = embeddings_q_c_l(emb_questions_test,emb_candidates_test,labels_test)
r_emb_question_candidate_pairs_dev = embeddings_q_c_l(emb_questions_dev,emb_candidates_dev,labels_dev)




        

In [ ]:
#pad the sentence based on given required length from the r_emb_question_candidate_pairs
def pad_sentence_embedding(sentence_embedding, required_length):

    if len(sentence_embedding) < required_length:
        for i in range(required_length - len(sentence_embedding)):
            sentence_embedding.append(np.zeros(len(sentence_embedding[0])))

    return sentence_embedding


# pad the question and candidate sentence embedding based on the max length of the question and candidate sentence
def get_pr_emb_question_candidate_pairs(r_emb_question_candidate_pairs):
  pr_emb_question_candidate_pairs = []
  for entry in r_emb_question_candidate_pairs:
      temp = []
      for question, candidate, label in entry:
          temp.append([pad_sentence_embedding(question, max_question_length), pad_sentence_embedding(candidate, max_candidate_length), label])
      pr_emb_question_candidate_pairs.append(temp)
  return pr_emb_question_candidate_pairs

pr_emb_question_candidate_pairs_train = get_pr_emb_question_candidate_pairs(r_emb_question_candidate_pairs_train)
pr_emb_question_candidate_pairs_test = get_pr_emb_question_candidate_pairs(r_emb_question_candidate_pairs_test)
pr_emb_question_candidate_pairs_dev = get_pr_emb_question_candidate_pairs(r_emb_question_candidate_pairs_dev)

In [ ]:
# change every list into a numpy array
for i in range(len(pr_emb_question_candidate_pairs_train)):
    for j in range(len(pr_emb_question_candidate_pairs_train[i])):
        pr_emb_question_candidate_pairs_train[i][j][0] = np.asarray(pr_emb_question_candidate_pairs_train[i][j][0], dtype=np.float32)
        pr_emb_question_candidate_pairs_train[i][j][1] = np.asarray(pr_emb_question_candidate_pairs_train[i][j][1], dtype=np.float32)

for i in range(len(pr_emb_question_candidate_pairs_test)):
    for j in range(len(pr_emb_question_candidate_pairs_test[i])):
        pr_emb_question_candidate_pairs_test[i][j][0] = np.asarray(pr_emb_question_candidate_pairs_test[i][j][0], dtype=np.float32)
        pr_emb_question_candidate_pairs_test[i][j][1] = np.asarray(pr_emb_question_candidate_pairs_test[i][j][1], dtype=np.float32)

for i in range(len(pr_emb_question_candidate_pairs_dev)):
    for j in range(len(pr_emb_question_candidate_pairs_dev[i])):
        pr_emb_question_candidate_pairs_dev[i][j][0] = np.asarray(pr_emb_question_candidate_pairs_dev[i][j][0], dtype=np.float32)
        pr_emb_question_candidate_pairs_dev[i][j][1] = np.asarray(pr_emb_question_candidate_pairs_dev[i][j][1], dtype=np.float32)

In [ ]:
# find the shape of the question and candidate sentence embedding
# print the number of questions and number of candidates for each question
print("Shape of question embedding: ", pr_emb_question_candidate_pairs_train[0][0][0].shape)
print("Shape of candidate embedding: ", pr_emb_question_candidate_pairs_train[0][0][1].shape)
print("Number of questions: ", len(pr_emb_question_candidate_pairs_train))
print("Number of Questions: ", len(pr_emb_question_candidate_pairs_train))

Shape of question embedding:  (23, 301)
Shape of candidate embedding:  (64, 301)
Number of questions:  867
Number of Questions:  867


In [ ]:
# save list as a pickle file and load
import pickle

def save_pickle(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f)

def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


In [ ]:
save_pickle(pr_emb_question_candidate_pairs_train,'./qc_embeddings_train_paper1.pkl')
save_pickle(pr_emb_question_candidate_pairs_test,'./qc_embeddings_test_paper1.pkl')
save_pickle(pr_emb_question_candidate_pairs_dev,'./qc_embeddings_dev_paper1.pkl')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load a pickle file to a list
import pickle
# load a pickle file to a list
import pickle
def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

pr_emb_question_candidate_pairs_train = load_pickle("qc_embeddings_train_paper1.pkl")
pr_emb_question_candidate_pairs_test = load_pickle("qc_embeddings_test_paper1.pkl")
pr_emb_question_candidate_pairs_dev = load_pickle("qc_embeddings_dev_paper1.pkl")

In [ ]:
import numpy as np

In [ ]:
# convert pr_emb_question_candidate_pairs to list of question_embedding, candidate_embedding, label

# list of question_embedding, candidate_embedding, label
def convert_to_qcl(pr_emb_question_candidate_pairs):
  qcl = []

  for i in range(len(pr_emb_question_candidate_pairs)):
      for j in range(len(pr_emb_question_candidate_pairs[i])):
          qcl.append([np.array(pr_emb_question_candidate_pairs[i][j][0]), np.array(pr_emb_question_candidate_pairs[i][j][1]), np.array(pr_emb_question_candidate_pairs[i][j][2])])
  return qcl

qcl_train = convert_to_qcl(pr_emb_question_candidate_pairs_train)
qcl_test = convert_to_qcl(pr_emb_question_candidate_pairs_test)
qcl_dev = convert_to_qcl(pr_emb_question_candidate_pairs_dev)

def seperate_qcl(qcl):
  questions_emb = []
  candidates_emb = []
  labels = []

  for i in range(len(qcl)):
      questions_emb.append(qcl[i][0])
      candidates_emb.append(qcl[i][1])
      labels.append(qcl[i][2])


  questions_emb = np.array(questions_emb)
  candidates_emb = np.array(candidates_emb)
  labels = np.array(labels)

  return questions_emb,candidates_emb,labels

questions_emb_train,candidates_emb_train,labels_train = seperate_qcl(qcl_train)
questions_emb_test,candidates_emb_test,labels_test = seperate_qcl(qcl_test)
questions_emb_dev,candidates_emb_dev,labels_dev = seperate_qcl(qcl_dev)


In [ ]:
labels_train.shape

(8592,)

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data_utils
import torch.nn.utils.rnn as rnn_utils
import random

class CNN_RNN(nn.Module):
    def __init__(self):
        super(CNN_RNN, self).__init__()
        # question size (301,32)
        # candidate size (301,64)
        
        # apply conv2d on question embedding with kernel size 5
        self.conv1 = nn.Conv2d(1, 1, 5)
        # apply conv2d on candidate embedding with kernel size 5
        self.conv2 = nn.Conv2d(1, 1, 5)

        # output size of conv1 = (301-5+1, 32-5+1) = (297, 28)
        # output size of conv2 = (301-5+1, 64-5+1) = (297, 60)

        # apply global max pooling along such that the size is (297,1)
        self.pool1 = nn.MaxPool2d((1, QUESTION_LEN-5+1))
        self.pool2 = nn.MaxPool2d((1, CANDIDATE_LEN-5+1))

        # output size of pool1 = (297,1)
        # output size of pool2 = (297,1)

        # concatenate the output of pool1 and pool2 
        # output size of concat = (297+297, 1) = (594, 1)

        # apply RNN on the concatenated output
        self.rnn = nn.RNN(594, 594, 1, batch_first=True)

        # classify the output of RNN as two classes
        self.fc = nn.Linear(594, 2)



    def forward(self,q,c):
        # q = q.view(-1, 1, 301, 32)
        # c = c.view(-1, 1, 301, 64)
        q = q.view(-1, 1, 301, QUESTION_LEN)
        c = c.view(-1, 1, 301, CANDIDATE_LEN)
        q = F.relu(self.conv1(q))
        c = F.relu(self.conv2(c))
        q = self.pool1(q)
        c = self.pool2(c)
        q = q.view(-1, 297)
        c = c.view(-1, 297)
        mul = torch.mul(q,c)
        sub = torch.sub(q,c)
        x = torch.cat((mul,sub),1)
        # x size = (1, 594)
        x = x.view(-1, 1, 594)
        x, _ = self.rnn(x)
        x = x.view(-1, 594)
        x = self.fc(x)
        # x size = (-1, 2)
        return x


class CNN_BiRNN(nn.Module):
    def __init__(self):
        super(CNN_BiRNN, self).__init__()
        # question size (301,32)
        # candidate size (301,64)
        
        # apply conv2d on question embedding with kernel size 5
        self.conv1 = nn.Conv2d(1, 1, 5)
        # apply conv2d on candidate embedding with kernel size 5
        self.conv2 = nn.Conv2d(1, 1, 5)

        # output size of conv1 = (301-5+1, 32-5+1) = (297, 28)
        # output size of conv2 = (301-5+1, 64-5+1) = (297, 60)

        # apply global max pooling along such that the size is (297,1)
        self.pool1 = nn.MaxPool2d((1, QUESTION_LEN-5+1))
        self.pool2 = nn.MaxPool2d((1, CANDIDATE_LEN-5+1))

        # output size of pool1 = (297,1)
        # output size of pool2 = (297,1)

        # concatenate the output of pool1 and pool2 
        # output size of concat = (297+297, 1) = (594, 1)

        # apply RNN on the concatenated output
        self.rnn = nn.RNN(594, 594, 1, batch_first=True,bidirectional=True)

        # classify the output of RNN as two classes
        self.fc = nn.Linear(2*594, 2)



    def forward(self,q,c):
        # q = q.view(-1, 1, 301, 32)
        # c = c.view(-1, 1, 301, 64)
        q = q.view(-1, 1, 301, QUESTION_LEN)
        c = c.view(-1, 1, 301, CANDIDATE_LEN)
        q = F.relu(self.conv1(q))
        c = F.relu(self.conv2(c))
        q = self.pool1(q)
        c = self.pool2(c)
        q = q.view(-1, 297)
        c = c.view(-1, 297)
        mul = torch.mul(q,c)
        sub = torch.sub(q,c)
        x = torch.cat((mul,sub),1)
        # x size = (1, 594)
        x = x.view(-1, 1, 594)
        x, _ = self.rnn(x)
        x = x.view(-1, 2*594)
        x = self.fc(x)
        # x size = (-1, 2)
        return x

class CNN_LSTM(nn.Module):
    def __init__(self):
        super(CNN_LSTM, self).__init__()
        # question size (301,32)
        # candidate size (301,64)
        
        # apply conv2d on question embedding with kernel size 5
        self.conv1 = nn.Conv2d(1, 1, 5)
        # apply conv2d on candidate embedding with kernel size 5
        self.conv2 = nn.Conv2d(1, 1, 5)

        # output size of conv1 = (301-5+1, 32-5+1) = (297, 28)
        # output size of conv2 = (301-5+1, 64-5+1) = (297, 60)

        # apply global max pooling along such that the size is (297,1)
        self.pool1 = nn.MaxPool2d((1, QUESTION_LEN-5+1))
        self.pool2 = nn.MaxPool2d((1, CANDIDATE_LEN-5+1))

        # output size of pool1 = (297,1)
        # output size of pool2 = (297,1)

        # concatenate the output of pool1 and pool2 
        # output size of concat = (297+297, 1) = (594, 1)

        # apply RNN on the concatenated output
        self.rnn = nn.LSTM(594, 594, 1, batch_first=True)

        # classify the output of RNN as two classes
        self.fc = nn.Linear(594, 2)



    def forward(self,q,c):
        # q = q.view(-1, 1, 301, 32)
        # c = c.view(-1, 1, 301, 64)
        q = q.view(-1, 1, 301, QUESTION_LEN)
        c = c.view(-1, 1, 301, CANDIDATE_LEN)
        q = F.relu(self.conv1(q))
        c = F.relu(self.conv2(c))
        q = self.pool1(q)
        c = self.pool2(c)
        q = q.view(-1, 297)
        c = c.view(-1, 297)
        mul = torch.mul(q,c)
        sub = torch.sub(q,c)
        x = torch.cat((mul,sub),1)
        # x size = (1, 594)
        x = x.view(-1, 1, 594)
        x, _ = self.rnn(x)
        x = x.view(-1, 594)
        x = self.fc(x)
        # x size = (-1, 2)
        return x


class CNN_BiLSTM(nn.Module):
    def __init__(self):
        super(CNN_BiLSTM, self).__init__()
        # question size (301,32)
        # candidate size (301,64)
        
        # apply conv2d on question embedding with kernel size 5
        self.conv1 = nn.Conv2d(1, 1, 5)
        # apply conv2d on candidate embedding with kernel size 5
        self.conv2 = nn.Conv2d(1, 1, 5)

        # output size of conv1 = (301-5+1, 32-5+1) = (297, 28)
        # output size of conv2 = (301-5+1, 64-5+1) = (297, 60)

        # apply global max pooling along such that the size is (297,1)
        self.pool1 = nn.MaxPool2d((1, QUESTION_LEN-5+1))
        self.pool2 = nn.MaxPool2d((1, CANDIDATE_LEN-5+1))

        # output size of pool1 = (297,1)
        # output size of pool2 = (297,1)

        # concatenate the output of pool1 and pool2 
        # output size of concat = (297+297, 1) = (594, 1)

        # apply RNN on the concatenated output
        self.rnn = nn.LSTM(594, 594, 1, batch_first=True, bidirectional=True)

        # classify the output of RNN as two classes
        self.fc = nn.Linear(2*594, 2)



    def forward(self,q,c):
        # q = q.view(-1, 1, 301, 32)
        # c = c.view(-1, 1, 301, 64)
        q = q.view(-1, 1, 301, QUESTION_LEN)
        c = c.view(-1, 1, 301, CANDIDATE_LEN)
        q = F.relu(self.conv1(q))
        c = F.relu(self.conv2(c))
        q = self.pool1(q)
        c = self.pool2(c)
        q = q.view(-1, 297)
        c = c.view(-1, 297)
        mul = torch.mul(q,c)
        sub = torch.sub(q,c)
        x = torch.cat((mul,sub),1)
        # x size = (1, 594)
        x = x.view(-1, 1, 594)
        x, _ = self.rnn(x)
        x = x.view(-1, 2*594)
        x = self.fc(x)
        # x size = (-1, 2)
        return x






In [ ]:
# train the model with batc size of 64
from tqdm import trange
def train(model,qcl,questions_emb,candidates_emb,labels,model_name,epochs=3):
  # define the model
  # define the loss function
  criterion = nn.CrossEntropyLoss()
  # define the optimizer
  optimizer = optim.Adam(model.parameters(), lr=0.001)

  # train the model on qcl ( question_embedding, candidate_embedding, label)

  for epoch in trange(epochs,desc='Epochs'):  # loop over the dataset multiple times
      running_loss = []
      for i in range(0,len(qcl),64):
      
          questions = questions_emb[i:i+64]
          candidates = candidates_emb[i:i+64]
          labels_64 = labels[i:i+64]


          # shape of question = (64, 301, 32)
          # shape of candidate = (64, 301, 64)
          # shape of label = (64,)
          
          # convert question to numpy array and change the size to (64,32,301)

          questions = questions.reshape(-1,QUESTION_LEN,301)
          # convert candidate to numpy array and change the size to (64,64,301)
          candidates = candidates.reshape(-1,CANDIDATE_LEN,301)

          # convert question to torch tensor
          questions = torch.from_numpy(questions)
          # convert candidates to torch tensor
          candidates = torch.from_numpy(candidates)
          # convert label to torch tensor

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = model(questions, candidates)
          loss = criterion(outputs, torch.from_numpy(labels_64))
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss.append(loss.item())

      # uncomment if you want to find loss for each epoch
      #print(f"Epoch : {epoch} , loss : {np.mean(running_loss)}")    
      torch.save(model.state_dict(),model_name+"_"+str(epoch)+".pt")
  return model





In [ ]:
def metric(model,pr_emb_question_candidate_pairs):
    model.eval()

    precisions = []
    ranks = []


    # extract question and candidate embeddings
    for q in pr_emb_question_candidate_pairs:
        real_labels = []
        predicted_scores = []
        for c_ind,c in enumerate(q):
            

            candidate_embedding = np.array(c[1])
            real_labels.append(c[2])
            # shape of qe = (1, 301, 32)
            # shape of c = (1, 301, 64)
            q_emb = np.array(c[0])
            q_emb = q_emb.reshape(1,QUESTION_LEN,301)
            candidate_embedding = candidate_embedding.reshape(1,CANDIDATE_LEN,301)
            # convert q_emb to torch tensor
            q_emb = torch.from_numpy(q_emb)
            # convert c to torch tensor
            candidate_embedding = torch.from_numpy(candidate_embedding)
            # forward
            outputs = model(q_emb, candidate_embedding)
            outputs = torch.nn.functional.softmax(outputs, dim=1)
            # convert outputs to numpy array
            outputs = outputs.detach().numpy()
            predicted_scores.append(outputs)

        
        real_labels = np.array(real_labels)
        question_real_labels_indices = np.where(real_labels == 1)[0]
        predicted_scores = np.array(predicted_scores)
        # print(predicted_scores.shape)
        predicted_scores = predicted_scores.squeeze(axis=1)
        # print(predicted_scores.shape)
        predicted_scores = predicted_scores[:,1] - predicted_scores[:,0]
        predicted_scores = np.array(predicted_scores)
        sorted_indices = np.argsort(predicted_scores)
        sorted_indices = sorted_indices[::-1]
        for i in range(len(sorted_indices)):
            if sorted_indices[i] in question_real_labels_indices:
                sorted_indices[i] = -1

        precision = 0
        rank = 0
        c = 0
        for i,ind in enumerate(sorted_indices):
            if ind == -1:
                rank = 1/(i+1)
                c += 1
                precision += (c/(i+1))
        
        
        
        precision = precision/c
        precisions.append(precision)
        ranks.append(rank)
    

        
    model.train()
    return np.mean(precisions), np.mean(ranks)

In [ ]:
# Training all models
models_names = ["CNN_RNN", "CNN_BiRNN", "CNN_LSTM", "CNN_BiLSTM"]
models = [CNN_RNN(), CNN_BiRNN(), CNN_LSTM(), CNN_BiLSTM()]

MAP_dict_train = {}
for model_name in models_names : 
  MAP_dict_train[model_name] = {}

MRR_dict_train = {}
for model_name in models_names : 
  MRR_dict_train[model_name] = {}

MAP_dict_test = {}
for model_name in models_names :
    MAP_dict_test[model_name] = {}

MRR_dict_test = {}
for model_name in models_names :
    MRR_dict_test[model_name] = {}

MAP_dict_dev = {}
for model_name in models_names :
    MAP_dict_dev[model_name] = {}

MRR_dict_dev = {}
for model_name in models_names :
    MRR_dict_dev[model_name] = {}


In [ ]:
for i in range(len(models)):
    train(models[i],qcl_train,questions_emb_train,candidates_emb_train,labels_train,models_names[i],epochs=10)

Epochs:  10%|█         | 1/10 [00:29<04:29, 29.94s/it]

Epoch : 0 , loss : 0.3542009859173386


Epochs:  20%|██        | 2/10 [00:55<03:40, 27.61s/it]

Epoch : 1 , loss : 0.32326814311522023


Epochs:  30%|███       | 3/10 [01:25<03:20, 28.57s/it]

...

Epoch : 7 , loss : 0.3064044333166546


Epochs:  90%|█████████ | 9/10 [05:52<00:39, 39.03s/it]

Epoch : 8 , loss : 0.30500713962095755


Epochs: 100%|██████████| 10/10 [06:31<00:00, 39.19s/it]

Epoch : 9 , loss : 0.3037599743516357


In [ ]:
for i in trange(10,desc="Computing train MAP and MRR for CNN_RNN"):
    model = CNN_RNN()
    model.load_state_dict(torch.load("CNN_RNN_"+str(i)+".pt"))
    precision, rank = metric(model,pr_emb_question_candidate_pairs_train)
    print("CNN_RNN_"+str(i)+".pt")
    
    MAP_dict_train['CNN_RNN'][i] = precision
    MRR_dict_train['CNN_RNN'][i] = rank

for i in trange(10,desc="Computing train MAP and MRR for CNN_BiRNN"):
    model = CNN_BiRNN()
    model.load_state_dict(torch.load("CNN_BiRNN_"+str(i)+".pt"))
    precision, rank = metric(model,pr_emb_question_candidate_pairs_train)
    print("CNN_BiRNN_"+str(i)+".pt")
    
    MAP_dict_train['CNN_BiRNN'][i] = precision
    MRR_dict_train['CNN_BiRNN'][i] = rank


for i in trange(10,desc="Computing train MAP and MRR for CNN_LSTM"):
    model = CNN_LSTM()
    model.load_state_dict(torch.load("CNN_LSTM_"+str(i)+".pt"))
    precision, rank = metric(model,pr_emb_question_candidate_pairs_train)
    
    
    MAP_dict_train['CNN_LSTM'][i] = precision
    MRR_dict_train['CNN_LSTM'][i] = rank

for i in trange(10,desc="Computing train MAP and MRR for CNN_BiLSTM"):
    model = CNN_BiLSTM()
    model.load_state_dict(torch.load("CNN_BiLSTM_"+str(i)+".pt"))
    precision, rank = metric(model,pr_emb_question_candidate_pairs_train)
    
    MAP_dict_train['CNN_BiLSTM'][i] = precision
    MRR_dict_train['CNN_BiLSTM'][i] = rank


Computing train MAP and MRR for CNN_RNN:  10%|█         | 1/10 [00:14<02:13, 14.87s/it]

CNN_RNN_0.pt


Computing train MAP and MRR for CNN_RNN:  20%|██        | 2/10 [00:28<01:51, 13.96s/it]

CNN_RNN_1.pt


Computing train MAP and MRR for CNN_RNN:  30%|███       | 3/10 [00:41<01:35, 13.64s/it]

...

Computing train MAP and MRR for CNN_BiRNN:  90%|█████████ | 9/10 [02:36<00:17, 17.43s/it]

CNN_BiRNN_8.pt


Computing train MAP and MRR for CNN_BiRNN: 100%|██████████| 10/10 [02:53<00:00, 17.33s/it]


CNN_BiRNN_9.pt


Computing train MAP and MRR for CNN_BiLSTM: 100%|██████████| 10/10 [06:19<00:00, 37.91s/it]


In [ ]:
for i in trange(10,desc="Computing dev MAP and MRR for CNN_RNN"):
    model = CNN_RNN()
    model.load_state_dict(torch.load("CNN_RNN_"+str(i)+".pt"))
    precision, rank = metric(model,pr_emb_question_candidate_pairs_dev)
    print("CNN_RNN_"+str(i)+".pt")
    
    MAP_dict_dev['CNN_RNN'][i] = precision
    MRR_dict_dev['CNN_RNN'][i] = rank

for i in trange(10,desc="Computing dev MAP and MRR for CNN_BiRNN"):
    model = CNN_BiRNN()
    model.load_state_dict(torch.load("CNN_BiRNN_"+str(i)+".pt"))
    precision, rank = metric(model,pr_emb_question_candidate_pairs_dev)
    print("CNN_BiRNN_"+str(i)+".pt")
    
    MAP_dict_dev['CNN_BiRNN'][i] = precision
    MRR_dict_dev['CNN_BiRNN'][i] = rank


for i in trange(10,desc="Computing dev MAP and MRR for CNN_LSTM"):
    model = CNN_LSTM()
    model.load_state_dict(torch.load("CNN_LSTM_"+str(i)+".pt"))
    precision, rank = metric(model,pr_emb_question_candidate_pairs_dev)
    
    
    MAP_dict_dev['CNN_LSTM'][i] = precision
    MRR_dict_dev['CNN_LSTM'][i] = rank

for i in trange(10,desc="Computing dev MAP and MRR for CNN_BiLSTM"):
    model = CNN_BiLSTM()
    model.load_state_dict(torch.load("CNN_BiLSTM_"+str(i)+".pt"))
    precision, rank = metric(model,pr_emb_question_candidate_pairs_dev)
    
    MAP_dict_dev['CNN_BiLSTM'][i] = precision
    MRR_dict_dev['CNN_BiLSTM'][i] = rank


Computing dev MAP and MRR for CNN_RNN:  10%|█         | 1/10 [00:02<00:19,  2.20s/it]

CNN_RNN_0.pt


Computing dev MAP and MRR for CNN_RNN:  20%|██        | 2/10 [00:04<00:16,  2.07s/it]

CNN_RNN_1.pt


Computing dev MAP and MRR for CNN_RNN:  30%|███       | 3/10 [00:06<00:14,  2.05s/it]

...

Computing dev MAP and MRR for CNN_BiRNN:  90%|█████████ | 9/10 [00:20<00:02,  2.29s/it]

CNN_BiRNN_8.pt


Computing dev MAP and MRR for CNN_BiRNN: 100%|██████████| 10/10 [00:23<00:00,  2.31s/it]


CNN_BiRNN_9.pt


Computing dev MAP and MRR for CNN_BiLSTM: 100%|██████████| 10/10 [00:51<00:00,  5.12s/it]


In [ ]:
for i in trange(10,desc="Computing test MAP and MRR for CNN_RNN"):
    model = CNN_RNN()
    model.load_state_dict(torch.load("CNN_RNN_"+str(i)+".pt"))
    precision, rank = metric(model,pr_emb_question_candidate_pairs_test)
    print("CNN_RNN_"+str(i)+".pt")
    
    MAP_dict_test['CNN_RNN'][i] = precision
    MRR_dict_test['CNN_RNN'][i] = rank

for i in trange(10,desc="Computing test MAP and MRR for CNN_BiRNN"):
    model = CNN_BiRNN()
    model.load_state_dict(torch.load("CNN_BiRNN_"+str(i)+".pt"))
    precision, rank = metric(model,pr_emb_question_candidate_pairs_test)
    print("CNN_BiRNN_"+str(i)+".pt")
    
    MAP_dict_test['CNN_BiRNN'][i] = precision
    MRR_dict_test['CNN_BiRNN'][i] = rank


for i in trange(10,desc="Computing test MAP and MRR for CNN_LSTM"):
    model = CNN_LSTM()
    model.load_state_dict(torch.load("CNN_LSTM_"+str(i)+".pt"))
    precision, rank = metric(model,pr_emb_question_candidate_pairs_test)
    
    
    MAP_dict_test['CNN_LSTM'][i] = precision
    MRR_dict_test['CNN_LSTM'][i] = rank

for i in trange(10,desc="Computing test MAP and MRR for CNN_BiLSTM"):
    model = CNN_BiLSTM()
    model.load_state_dict(torch.load("CNN_BiLSTM_"+str(i)+".pt"))
    precision, rank = metric(model,pr_emb_question_candidate_pairs_test)
    
    MAP_dict_test['CNN_BiLSTM'][i] = precision
    MRR_dict_test['CNN_BiLSTM'][i] = rank


Computing test MAP and MRR for CNN_RNN:  10%|█         | 1/10 [00:04<00:39,  4.42s/it]

CNN_RNN_0.pt


Computing test MAP and MRR for CNN_RNN:  20%|██        | 2/10 [00:08<00:32,  4.01s/it]

CNN_RNN_1.pt


Computing test MAP and MRR for CNN_RNN:  30%|███       | 3/10 [00:11<00:26,  3.83s/it]

...

Computing test MAP and MRR for CNN_BiRNN:  90%|█████████ | 9/10 [00:41<00:04,  4.69s/it]

CNN_BiRNN_8.pt


Computing test MAP and MRR for CNN_BiRNN: 100%|██████████| 10/10 [00:45<00:00,  4.59s/it]


CNN_BiRNN_9.pt


Computing test MAP and MRR for CNN_BiLSTM: 100%|██████████| 10/10 [01:44<00:00, 10.44s/it]


In [ ]:
import pickle



try:
  geeky_file = open('MAP_dict_test', 'wb')
  pickle.dump(MAP_dict_test, geeky_file)
  geeky_file.close()
  
 

  geeky_file = open('MAP_dict_train', 'wb')
  pickle.dump(MAP_dict_train, geeky_file)
  geeky_file.close()

  geeky_file = open('MAP_dict_dev', 'wb')
  pickle.dump(MAP_dict_dev, geeky_file)
  geeky_file.close()

  geeky_file = open('MRR_dict_test', 'wb')
  pickle.dump(MRR_dict_test, geeky_file)
  geeky_file.close()
  
 

  geeky_file = open('MRR_dict_train', 'wb')
  pickle.dump(MRR_dict_train, geeky_file)
  geeky_file.close()

  geeky_file = open('MRR_dict_dev', 'wb')
  pickle.dump(MRR_dict_dev, geeky_file)
  geeky_file.close()
	

	

except:
	print("Something went wrong")
 


In [ ]:
MRR_dict_dev

{'CNN_RNN': {0: 0.6464378954378954,
  1: 0.6692549737843856,
  2: 0.6756396779690896,
  3: 0.6722350094350095,
  4: 0.6689560620665884,
  5: 0.665622728733255,
  6: 0.6709188478188479,
  7: 0.6756116582763642,
  8: 0.671871687136393,
  9: 0.6695097823744882},
 'CNN_BiRNN': {0: 0.6370610089257146,
  1: 0.6431658263305321,
  2: 0.6446063492063493,
  3: 0.6432148629148629,
  4: 0.6310359562006621,
  5: 0.6287855144855145,
  6: 0.6367024864024863,
  7: 0.6416628038628038,
  8: 0.6362437562437563,
  9: 0.6391632922632922},
 'CNN_LSTM': {0: 0.6191395966606493,
  1: 0.6370116176221439,
  2: 0.6387033925139187,
  3: 0.6522914711019975,
  4: 0.6809961623172148,
  5: 0.683335715746242,
  6: 0.6900503307150366,
  7: 0.6965299589299588,
  8: 0.6952350094350094,
  9: 0.6942778665778665},
 'CNN_BiLSTM': {0: 0.6178405483405484,
  1: 0.6419860028860029,
  2: 0.6533095825742884,
  3: 0.6512458019105078,
  4: 0.6546224100976421,
  5: 0.6485308350060671,
  6: 0.6628343388637506,
  7: 0.664719306184012,
 

In [ ]:
with open('MRR_dict_dev', 'rb') as handle:
    temp_dict = pickle.load(handle)

In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
import pickle

def load_pickle(file):
  with open(file,'rb') as f:
    return pickle.load(f)

In [ ]:
MAP_dict_train = load_pickle('MAP_dict_train')
MRR_dict_train = load_pickle('MRR_dict_train')
MAP_dict_dev = load_pickle('MAP_dict_dev')
MRR_dict_dev = load_pickle('MRR_dict_dev')
MAP_dict_test = load_pickle('MAP_dict_test')
MRR_dict_test = load_pickle('MRR_dict_test')

In [ ]:
MAP_dict_test

{'CNN_RNN': {0: 0.6253154691648091,
  1: 0.6445816840291366,
  2: 0.6543541742258175,
  3: 0.6529217330912183,
  4: 0.6550247621610499,
  5: 0.6608472558449161,
  6: 0.6603069431514839,
  7: 0.6558958132881019,
  8: 0.6514771144947431,
  9: 0.6556331691447046},
 'CNN_BiRNN': {0: 0.636338470902986,
  1: 0.6388376934181824,
  2: 0.6539765717197725,
  3: 0.6529791975260226,
  4: 0.6449462987104507,
  5: 0.6438752771519508,
  6: 0.649421364637589,
  7: 0.6447754684637271,
  8: 0.6397086696675736,
  9: 0.640077015855665},
 'CNN_LSTM': {0: 0.6138280938190247,
  1: 0.6316922740879468,
  2: 0.640958482948821,
  3: 0.6484526911851242,
  4: 0.657938262200936,
  5: 0.6605455804986308,
  6: 0.6636536184393047,
  7: 0.6660470503120639,
  8: 0.6697212244153835,
  9: 0.6679859345186366},
 'CNN_BiLSTM': {0: 0.6222211928098572,
  1: 0.645006513722024,
  2: 0.6482745245497522,
  3: 0.6504861449022167,
  4: 0.6668514160792035,
  5: 0.6684986642957645,
  6: 0.6724546409086448,
  7: 0.6726266388316887,
  8

In [ ]:
def metrics(model_name):
  for e in range(10):
    print("MRR (train): ",MRR_dict_train[model_name][e],"MAP (train): ",MAP_dict_train[model_name][e],"MRR (dev): ",MRR_dict_dev[model_name][e],"MAP (dev): ",MAP_dict_dev[model_name][e],"MRR (test): ",MRR_dict_test[model_name][e],"MAP (test): ",MAP_dict_test[model_name][e])

In [ ]:
metrics('CNN_BiLSTM')

MRR (train):  0.5614359841637723 MAP (train):  0.6161056796531179 MRR (dev):  0.6178405483405484 MAP (dev):  0.6443849927849928 MRR (test):  0.5683005188185765 MAP (test):  0.6222211928098572
MRR (train):  0.5924844857052279 MAP (train):  0.6494269178725736 MRR (dev):  0.6419860028860029 MAP (dev):  0.6677526695526695 MRR (test):  0.5887389146858427 MAP (test):  0.645006513722024
MRR (train):  0.5940160797318942 MAP (train):  0.6519165946768152 MRR (dev):  0.6533095825742884 MAP (dev):  0.6817429159076217 MRR (test):  0.591990321940482 MAP (test):  0.6482745245497522
MRR (train):  0.5967267484763757 MAP (train):  0.655527462280797 MRR (dev):  0.6512458019105078 MAP (dev):  0.681679135243841 MRR (test):  0.5933007833635449 MAP (test):  0.6504861449022167
MRR (train):  0.595154403283838 MAP (train):  0.654398853703523 MRR (dev):  0.6546224100976421 MAP (dev):  0.6852557434309756 MRR (test):  0.6094512298487195 MAP (test):  0.6668514160792035
MRR (train):  0.5974357260565972 MAP (train): 

In [ ]:
temp_dict

{'CNN_RNN': {0: 0.6464378954378954,
  1: 0.6692549737843856,
  2: 0.6756396779690896,
  3: 0.6722350094350095,
  4: 0.6689560620665884,
  5: 0.665622728733255,
  6: 0.6709188478188479,
  7: 0.6756116582763642,
  8: 0.671871687136393,
  9: 0.6695097823744882},
 'CNN_BiRNN': {0: 0.6370610089257146,
  1: 0.6431658263305321,
  2: 0.6446063492063493,
  3: 0.6432148629148629,
  4: 0.6310359562006621,
  5: 0.6287855144855145,
  6: 0.6367024864024863,
  7: 0.6416628038628038,
  8: 0.6362437562437563,
  9: 0.6391632922632922},
 'CNN_LSTM': {0: 0.6191395966606493,
  1: 0.6370116176221439,
  2: 0.6387033925139187,
  3: 0.6522914711019975,
  4: 0.6809961623172148,
  5: 0.683335715746242,
  6: 0.6900503307150366,
  7: 0.6965299589299588,
  8: 0.6952350094350094,
  9: 0.6942778665778665},
 'CNN_BiLSTM': {0: 0.6178405483405484,
  1: 0.6419860028860029,
  2: 0.6533095825742884,
  3: 0.6512458019105078,
  4: 0.6546224100976421,
  5: 0.6485308350060671,
  6: 0.6628343388637506,
  7: 0.664719306184012,
 